In [1]:
import pandas as pd
import numpy as np

# 可視化用のライブラリ
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import neologdn
import MeCab

import re

from tqdm import tqdm_notebook as tqdm
from tqdm._tqdm_notebook import tqdm_notebook
# # プログレスバーのラベル設定
tqdm_notebook.pandas(desc="progress: ")

#前処理用ライブラリ
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

# 機械学習モデル関連ライブラリ
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn import neighbors


# モデル評価関連ライブラリ
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc, roc_curve

In [3]:
# 内容_id×全品詞の単語のデータフレームの読み込み
std001_model_tfidf_df = pd.read_pickle('std001_model_tfidf_df.pkl')
std001_model_tfidf_df.head()

original_type,以上_名詞-非自立-副詞可能,と_助詞-格助詞-引用,・_記号-一般,気_名詞-非自立-一般,これ_名詞-代名詞-一般,悪い_形容詞-自立,いう_動詞-自立,で_助詞-格助詞-一般,てる_動詞-非自立,ずっと_副詞-一般,...,ある_助動詞,から_助詞-接続助詞,事_名詞-非自立-一般,が_助詞-接続助詞,として_助詞-格助詞-連語,業務_名詞-一般,変わる_動詞-自立,で_助詞-接続助詞,悩む_動詞-自立,ので_助詞-接続助詞
内容_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.705146,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# [内容_id]をインデックスからカラムに移動
std001_model_tfidf_df = std001_model_tfidf_df.reset_index()
std001_model_tfidf_df.head()

original_type,内容_id,以上_名詞-非自立-副詞可能,と_助詞-格助詞-引用,・_記号-一般,気_名詞-非自立-一般,これ_名詞-代名詞-一般,悪い_形容詞-自立,いう_動詞-自立,で_助詞-格助詞-一般,てる_動詞-非自立,...,ある_助動詞,から_助詞-接続助詞,事_名詞-非自立-一般,が_助詞-接続助詞,として_助詞-格助詞-連語,業務_名詞-一般,変わる_動詞-自立,で_助詞-接続助詞,悩む_動詞-自立,ので_助詞-接続助詞
0,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.705146,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 次元削減のための準備
DataFrameからarray型へ変換

In [7]:
std001_model_tfidf_df_array = std001_model_tfidf_df.iloc[:, 1:].values# [内容]カラムのみarray型にしないため除外
std001_model_tfidf_df_array

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.24816917,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

### 次元圧縮
次元圧縮法として代表的な手法は3つ

### 1.NMFで次元削減

In [15]:
# 2次元に変換
from sklearn.decomposition import NMF
model = NMF(n_components=2, init='random', random_state=0)
X_mnf = model.fit_transform(std001_model_tfidf_df_array)
X_mnf[:2]

array([[0.03123923, 0.10567819],
       [0.        , 0.20417421]])

In [27]:
tmp_df = pd.DataFrame(X_mnf, columns=['vec1', 'vec2'])
tmp_df.columns[tmp_df.nunique() == 1]

Index([], dtype='object')

### 2.PCAで次元削減

In [10]:
# まずはStandardScaler を利用し数値を標準化(平均が 0 で標準偏差・分散が 1 )
# StandardScalerを使用すると負の値が出てくるのでtandardScaler後のNMFでの次元削減はできない
from sklearn.preprocessing import StandardScaler
sds = StandardScaler()
X_sds = sds.fit_transform(std001_model_tfidf_df_array)
X_sds

array([[-0.18428854, -0.23435016, -0.19743784, ..., -0.20401361,
        -0.20402659, -0.20393982],
       [-0.18428854, -0.23435016, -0.19743784, ..., -0.20401361,
        -0.20402659, -0.20393982],
       [-0.18428854, -0.23435016, -0.19743784, ..., -0.20401361,
        -0.20402659, -0.20393982],
       ...,
       [-0.18428854, -0.23435016, -0.19743784, ..., -0.20401361,
         4.74207254, -0.20393982],
       [-0.18428854, -0.23435016, -0.19743784, ..., -0.20401361,
        -0.20402659, -0.20393982],
       [-0.18428854, -0.23435016, -0.19743784, ..., -0.20401361,
        -0.20402659, -0.20393982]])

In [12]:
# pca
# 2次元に変換
from sklearn.decomposition import PCA
model = PCA(n_components=2)
X_pca = model.fit_transform(X_sds)
X_pca[:2]

array([[-1.09758459, -0.50636748],
       [ 1.0067593 ,  0.95176175]])

### 3.t-SNEで次元圧縮

In [14]:
# StandardScaler利用後の値を使用
# 2次元に変換
from sklearn.manifold import TSNE
X_sne = TSNE(n_components=2).fit_transform(X_sds)
X_sne[:2]

array([[ 88.81577 , -43.58643 ],
       [-60.87027 , -29.409252]], dtype=float32)

## 次元圧縮後の単語の値でクラスタリング
今回はコサイン類似度を基準とした階層型クラスタリングを行って2つのクラスタに分類

In [17]:
# PCAによる次元圧縮後の値を使用
# NMFによる次元削減後の値には２次元どちらも「０」であるレコードがあるためエラーが発生してしまうらしい
from sklearn.cluster import AgglomerativeClustering
model = AgglomerativeClustering(n_clusters=2, linkage="average", affinity="cosine")
y = model.fit_predict(X_pca)
y

array([0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 0])

In [22]:
# 念のため
# t-sneによる次元圧縮後の値を使用
from sklearn.cluster import AgglomerativeClustering
model = AgglomerativeClustering(n_clusters=2, linkage="average", affinity="cosine")
y2 = model.fit_predict(X_sne)
y2

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0])

## 次元削減後の値とクラスタ番号を元データフレームと連結

In [18]:
# まずはPCAによる次元圧縮後の値をデータフレームにしておく
dimension_reduction_df = pd.DataFrame(X_pca, columns=['vec1', 'vec2'])
dimension_reduction_df.head()

,vec1,vec2
0,-1.097585,-0.506367
1,1.006759,0.951762
2,-1.047338,0.584942
3,-1.768277,-2.735284
4,-1.609191,0.022630


In [19]:
# 次にPCAによる次元圧縮後の値のみのデータフレームを[内容_id]カラムのみのデータフレームとインデックス番号をキーにして紐付ける
# index をキーに結合したい場合は、DataFrame.joinを使用
dimension_reduction_join_df = std001_model_tfidf_df[['内容_id']].join(dimension_reduction_df)
dimension_reduction_join_df.head()

,内容_id,vec1,vec2
0,0,-1.097585,-0.506367
1,1,1.006759,0.951762
2,2,-1.047338,0.584942
3,3,-1.768277,-2.735284
4,4,-1.609191,0.022630


In [20]:
# 加えて、クラスタリング番号の値も紐付ける
dimension_reduction_join_df['cluster'] = y
dimension_reduction_join_df.head()

,内容_id,vec1,vec2,cluster
0,0,-1.097585,-0.506367,0
1,1,1.006759,0.951762,0
2,2,-1.047338,0.584942,0
3,3,-1.768277,-2.735284,1
4,4,-1.609191,0.022630,0
